In [2]:
!pip install jaxlib jax --upgrade 
!pip install neural-tangents 
!pip install tensorflow tensorflow-datasets --upgrade

Requirement already up-to-date: jaxlib in /home/udit/anaconda3/envs/synflow-py3.7/lib/python3.7/site-packages (0.1.52)
Requirement already up-to-date: jax in /home/udit/anaconda3/envs/synflow-py3.7/lib/python3.7/site-packages (0.1.75)
     |████████████████████████████████| 97 kB 6.6 MB/s  eta 0:00:01
Processing /home/udit/.cache/pip/wheels/68/17/69/ac196dd181e620bba5fae5488e4fd6366a7316dce13cf88776/frozendict-1.2-py3-none-any.whl
  Using cached tensorflow-2.3.0-cp37-cp37m-manylinux2010_x86_64.whl (320.4 MB)
     |████████████████████████████████| 3.4 MB 14.0 MB/s eta 0:00:01
  Using cached numpy-1.18.5-cp37-cp37m-manylinux1_x86_64.whl (20.1 MB)
  Using cached tensorflow_estimator-2.3.0-py2.py3-none-any.whl (459 kB)
  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached tensorboard-2.3.0-py3-none-any.whl (6.8 MB)
  Using cached scipy-1.4.1-cp37-cp37m-manylinux1_x86_64.whl (26.1 MB)
Processing /home/udit/.cache/p

  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21495 sha256=f16641c7f0f77036295d69029582ba116f0534ff708094c85a8c09b3bb7b3fbd
  Stored in directory: /home/udit/.cache/pip/wheels/29/93/c6/762e359f8cb6a5b69c72235d798804cae523bbe41c2aa8333d
Successfully built dill promise
ERROR: tensorflow-metadata 0.23.0 has requirement absl-py<0.9,>=0.7, but you'll have absl-py 0.10.0 which is incompatible.
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.0
    Uninstalling numpy-1.19.0:
      Successfully uninstalled numpy-1.19.0
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.2
    Uninstalling scipy-1.5.2:
      Successfully uninstalled scipy-1.5.2


In [1]:
import time
import itertools

import numpy.random as npr

import jax.numpy as np
from jax import jit, grad, random
from jax.experimental import optimizers
from neural_tangents import stax
from neural_tangents.stax import Dense, Relu
from examples import datasets
import neural_tangents as nt

In [26]:
rng = random.PRNGKey(0)

batch_size = 32

train_images, train_labels, test_images, test_labels = datasets.get_dataset("mnist")
num_train = train_images.shape[0]
num_complete_batches, leftover = divmod(num_train, batch_size)
num_batches = num_complete_batches + bool(leftover)

In [27]:
def data_stream():
    rng = npr.RandomState(0)

    while True:
        perm = rng.permutation(num_train)
        for i in range(num_batches):
            batch_idx = perm[i * batch_size:(i + 1) * batch_size]
            yield train_images[batch_idx], train_labels[batch_idx]

batches = data_stream()

In [30]:
learning_rate = 0.01
num_epochs = 10 # 10000

opt_init, opt_update, get_params = optimizers.sgd(learning_rate)
opt_update = jit(opt_update)

def loss(params, batch, fn):
    inputs, targets = batch
    preds = fn(params, inputs)
#    print(preds.shape, targets.shape)
    return 0.5 * np.mean((preds - targets) ** 2)
    # return preds * targets

@jit
def update(i, opt_state, batch):
    params = get_params(opt_state)
    return opt_update(i, grad(loss)(params, batch, apply_fn), opt_state)

init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Dense(1000), stax.Relu(),
    stax.Dense(1000), stax.Relu(),
    stax.Dense(10)
)

apply_fn = jit(apply_fn)
# kernel_fn = jit(kernel_fn)

_, init_params = init_fn(rng, (-1, 28 * 28))
opt_state = opt_init(init_params)

itercount = itertools.count()
kernel_fn_emp = nt.empirical_kernel_fn(apply_fn, diagonal_axes=()) #trace_axes=(),

print("\nStarting training...")
for epoch in range(num_epochs):
    start_time = time.time()
    for _ in range(num_batches):
        opt_state = update(next(itercount), opt_state, next(batches))
    epoch_time = time.time() - start_time
    perm = npr.RandomState(0).permutation(num_train)
    for i in range(1):
        batch_idx = perm[i * batch_size:(i + 1) * batch_size]
        x_train = train_images[batch_idx]
        start_time = time.time()
        ntk = kernel_fn_emp
        kernel = ntk(x_train, x_train, 'ntk', get_params(opt_state))
        # print(kernel)
        print(kernel.shape)
        print(np.linalg.norm(kernel))
        epoch_time_ntk = time.time() - start_time
        print("NTK: Epoch {} in {:0.2f} sec".format(epoch, epoch_time_ntk))
        # print(np.linalg.norm(kernel_fn_emp(train_images, train_images, 'ntk', get_params(opt_state))))
    print("Epoch {} in {:0.2f} sec".format(epoch, epoch_time))


Starting training...
(32, 32)
9.133637
NTK: Epoch 0 in 2.33 sec
Epoch 0 in 5.68 sec
(32, 32)
9.097219
NTK: Epoch 1 in 1.52 sec
Epoch 1 in 5.50 sec
(32, 32)
9.083096
NTK: Epoch 2 in 1.53 sec
Epoch 2 in 5.52 sec
(32, 32)
9.060886
NTK: Epoch 3 in 1.51 sec
Epoch 3 in 5.54 sec
(32, 32)
9.047582
NTK: Epoch 4 in 1.51 sec
Epoch 4 in 5.53 sec
(32, 32)
9.0311
NTK: Epoch 5 in 1.50 sec
Epoch 5 in 5.54 sec
(32, 32)
9.01098
NTK: Epoch 6 in 1.49 sec
Epoch 6 in 5.55 sec
(32, 32)
8.999
NTK: Epoch 7 in 1.51 sec
Epoch 7 in 5.54 sec
(32, 32)
8.979445
NTK: Epoch 8 in 1.50 sec
Epoch 8 in 5.53 sec
(32, 32)
8.970364
NTK: Epoch 9 in 1.50 sec
Epoch 9 in 5.50 sec


In [9]:
x = 10
print(x)

10


In [40]:
Starting training...
(32, 32, 10, 10)
29.026665
NTK: Epoch 0 in 2.35 sec
Epoch 0 in 5.67 sec
(32, 32, 10, 10)
28.9128
NTK: Epoch 1 in 1.50 sec
Epoch 1 in 5.51 sec
(32, 32, 10, 10)
28.868113
NTK: Epoch 2 in 1.53 sec
Epoch 2 in 5.51 sec
(32, 32, 10, 10)
28.797829
NTK: Epoch 3 in 1.50 sec
Epoch 3 in 5.53 sec
(32, 32, 10, 10)
28.755636
NTK: Epoch 4 in 1.52 sec
Epoch 4 in 5.51 sec
(32, 32, 10, 10)
28.705595
NTK: Epoch 5 in 1.52 sec
Epoch 5 in 5.65 sec
(32, 32, 10, 10)
28.639294
NTK: Epoch 6 in 1.52 sec
Epoch 6 in 5.50 sec
(32, 32, 10, 10)
28.601112
NTK: Epoch 7 in 1.53 sec
Epoch 7 in 5.48 sec
(32, 32, 10, 10)
28.539501
NTK: Epoch 8 in 1.52 sec
Epoch 8 in 5.50 sec
(32, 32, 10, 10)
28.510082
NTK: Epoch 9 in 1.53 sec
Epoch 9 in 5.48 sec


NameError: name 'torch' is not defined